In [34]:
import pandas as pd
import numpy as np
import pingouin as pg
from scipy import stats
from statsmodels.stats import weightstats as ws
from pydataset import data

In [35]:
df = data('iris')

# T-test

Assumptions: The population that samples are taken is approximately normal. In this case, the normalized version of $\bar{X}$ using the sample standard deviation $S$ instead of the population standard deviation $\sigma$ follows the $t$-distribution with $n-1$ Degree of Freedom (dof).
- It is robust to the normality assumption

Test statistic: $T=\frac{\bar{X}-\mu_0}{S/\sqrt{n}}$



## 1-sample *t*-test

Population: Sepal.Length of all 3 species (without differentiation among species).
- $H_0$: $\mu=5$
- $H_a$: $\mu\neq5$

In [44]:
pg.ttest(
    x=df['Sepal.Length'], 
    y=5, 
    paired=False,
    tail='two-sided', #'greater', 'less'
    correction='auto', #True, False
    confidence=0.95
)

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,12.473257,149,two-sided,6.670742e-25,"[5.71, 5.98]",1.018437,5.88e+21,1.0


## 2-sample *t*-test

- Population 1: Sepal.Length of setosa
- Population 2: Sepal.Length of versicolor
- $H_0$: $D_0 = \mu_1 − \mu_2 = 0$
- $H_a$: $D_a = \mu_1 − \mu_2 \neq 0$

In [53]:
pg.ttest(
    x=df.loc[df['Species']=='setosa','Sepal.Length'], 
    y=df.loc[df['Species']=='versicolor','Sepal.Length'],
    paired=False, # True for paired samples
    tail='two-sided', #'greater', 'less'
    correction='auto', #True, False
    confidence=0.95
)

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-10.520986,98,two-sided,8.985235e-18,"[-1.11, -0.75]",2.104197,4.19e+14,1.0


# *Z*-Test

Assumptions: When sample size is large, the distribution of sample mean $(\bar{X})$ is approximately normal with
- $E(\bar{X}) = \mu$ 
- $Var(\bar{X}) = \sigma^2/n$
- where $\mu$ and $\sigma^2$ are the population mean and variance, respectively.

Test statistic: $Z=\frac{\bar{X}-\mu_0}{\sigma/\sqrt{N}}$

## 1-sample *Z*-test

Population: Sepal.Length of all 3 species (without differentiation among species).
- $H_0$: $\mu=5$
- $H_a$: $\mu\neq5$

In [36]:
zstat, pval = ws.ztest(
    x1=df['Sepal.Length'], 
    x2=None, 
    value=5,
    alternative='two-sided' #'larger', 'smaller'
)
print("z-stat:", ztest, "p-value:", pval)

z-stat: 0.04930141164702255 p-value: 1.0446696695008141e-35


## 2-sample *Z*-test

- Population 1: Sepal.Length of setosa
- Population 2: Sepal.Length of versicolor
- $H_0$: $D_0 = \mu_1 − \mu_2 = 0$
- $H_a$: $D_a = \mu_1 − \mu_2 \neq 0$

In [37]:
zstat, pval = ws.ztest(
    x1=df.loc[df['Species']=='setosa','Sepal.Length'], 
    x2=df.loc[df['Species']=='versicolor','Sepal.Length'],
    value=0,
    alternative='two-sided' #'larger', 'smaller'
)
print("z-stat:", ztest, "p-value:", pval)

z-stat: 0.04930141164702255 p-value: 6.914595261207391e-26
